In [1]:
# 这个文件用来处理数据
# 导入必要的库
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
import pyarrow
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 

In [52]:
# 这里需要处理所有6类数据

def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    # implement here all desired dtypes for tables
    # the following is just an example
    for col in df.columns:
        # last letter of column name will help you determine the type
        if col[-1] in ("P", "A"):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
            
        # 处理D(日期) 转为时间戳
        if col[-1] == 'D':
            # 0 if null, otherwise timestamp.  
            df = df.with_columns(
                pl.when(pl.col(col).is_null())
                    .then(pl.lit(0))
                    .otherwise(pl.col(col).str.to_date( ).dt.timestamp())
                    .alias(col)
            )
            #print(df[col])

    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

def remove_letters(text):
    text = str(text)
    return int("".join(filter(str.isdigit, text)))

def employment_length(text):
    text = str(text)
    if text == "LESS_ONE":
        return 1
    if text == "MORE_ONE":
        return 2
    if text == "MORE_FIVE":
        return 3
    return 0
    

In [32]:
#  先考虑训练集

# base
train_basetable = pl.read_csv("train/train_base.csv")

# static
train_static = pl.concat(
    [
        pl.read_csv("train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv("train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)

# static_cb
train_static_cb = pl.read_csv("train/train_static_cb_0.csv").pipe(set_table_dtypes)

# person 这里添加了新文件；数字对应depth
# 例：person1 depth=1; person2 depth=2
train_person_1 = pl.read_csv("train/train_person_1.csv").pipe(set_table_dtypes)
train_person_2 = pl.read_csv("train/train_person_2.csv").pipe(set_table_dtypes)

# credit_bureau 这里添加了新文件；数字对应depth
train_credit_bureau_b_1 = pl.read_csv("train/train_credit_bureau_b_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv("train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [67]:
# (1) 先处理train_person_1

train_person_1_feats = train_person_1.group_by("case_id").agg(
    # 1.0 baseline
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed"),
    # 1.1 出生日期 birthdate
    pl.col("birth_259D").first().alias("birth_259D_first"),
    # 1.2 孩子数量 # children
    pl.col("childnum_185L").first().alias("childnum_185L_first"),
    # 1.3 联系人地址邮编 zip code
    pl.col("contaddr_district_15M").first().alias("contaddr_district_15M_string"),
    # 1.4 can match contact address
    pl.col("contaddr_matchlist_1032L").first().alias("contaddr_matchlist_1032L_ismatch"),
    # 1.5 联系地址与工作地址是否相同 addr same with work addr
    pl.col("contaddr_smempladdr_334L").first().alias("contaddr_smempladdr_334L_issame"),
    # 1.7 contact address
    pl.col("contaddr_zipcode_807M").first().alias("contaddr_zipcode_807M_string"),
    # 1.8 education level
    pl.col("education_927M").first().map_elements(remove_letters, pl.Int32).alias("education_927M_first"),
    # 1.9 employment start date
    pl.col("empl_employedfrom_271D").first().alias("empl_employedfrom_271D_first"),
    # 1.10 employment length
    pl.col("empl_employedtotal_800L").first().alias("empl_employedtotal_800L_string"),
    # 1.11 employment industry
    pl.col("empl_industry_691L").first().alias("empl_industry_691L_string"),
    # 1.12 employment district
    pl.col("empladdr_district_926M").first().alias("empladdr_district_926M_string"),
    # 1.13 employment zipcode
    pl.col("empladdr_zipcode_114M").first().alias("empladdr_zipcode_114M_string"),
    # 1.14 family state
    pl.col("familystate_447L").first().alias("familystate_447L_string"),
    # 1.15 gender
    pl.col("gender_992L").first().alias("gender_992L_string"),
    # 1.16 type of housing
    pl.col("housingtype_772L").first().alias("housingtype_772L_string"),
    # 1.17 type of income
    pl.col("incometype_1044T").first().alias("incometype_1044T_string"),
    # 1.18 is reference?
    pl.col("isreference_387L").first().alias("isreference_387L_string"),
    # 1.19 primary language
    pl.col("language1_981M").first().alias("language1_981M_string"),
    # 1.20 amount of main income
    pl.col("mainoccupationinc_384A").first().alias("mainoccupationinc_384A_number"),
)

#
#pl.col("").first().alias(""),

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

train_person_1_feats.shape

(1526659, 22)

In [68]:
# We will process in this examples only A-type and M-type columns, so we need to select them.
# 这里D类型也包括在内
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M", "D"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M", "D"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'datefirstoffer_1144D', 'datelastinstal40dpd_247D', 'datelastunpaid_3546854D', 'disbursedcredamount_1113A', 'downpmt_116A', 'dtlastpmtallstes_4499206D', 'firstclxcampaign_1125D', 'firstdatedue_489D', 'inittransactionamount_650A', 'lastactivateddate_801D', 'lastapplicationdate_877D', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastapprdate_640D', 'lastcancelreason_561M', 'lastdelinqdate_224D', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectdate_50D', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastrepayingdate_696D', 'maininc_215A', 'maxannuity_159A', 'maxannuity_

In [84]:

case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)
    #cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'datefirstoffer_1144D', 'datelastinstal40dpd_247D', 'datelastunpaid_3546854D', 'disbursedcredamount_1113A', 'downpmt_116A', 'dtlastpmtallstes_4499206D', 'firstclxcampaign_1125D', 'firstdatedue_489D', 'inittransactionamount_650A', 'lastactivateddate_801D', 'lastapplicationdate_877D', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastapprdate_640D', 'lastcancelreason_561M', 'lastdelinqdate_224D', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectdate_50D', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastrepayingdate_696D', 'maininc_215A', 'maxannuity_159A', 'maxannuity_

In [85]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")
X_train

Train: (915995, 72)
Valid: (305332, 72)
Test: (305332, 72)


,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,credamount_770A,currdebt_22A,currdebtcredtyperange_828A,...,education_88M,maritalst_385M,maritalst_893M,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtssum_45A,responsedate_1012D,responsedate_4527233D,responsedate_4917613D
0,NaN,1917.6000,0.0,NaN,NaN,NaN,NaN,30000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,4937.0000,0.0,NaN,NaN,NaN,NaN,78000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3600.0000,0.0,NaN,NaN,NaN,NaN,60000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3110.8000,0.0,NaN,NaN,NaN,NaN,20000.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1218.0000,0.0,NaN,NaN,NaN,NaN,20300.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915990,104557.25,8218.0000,5293.2,8027.2000,NaN,23399.828,5293.2,132000.0,10548.4,10548.4,...,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,0.0,0.0,1.603066e+15
915991,176561.36,3675.4001,0.0,7356.8003,NaN,16392.496,6750.2,30000.0,0.0,0.0,...,a55475b1,a55475b1,a55475b1,NaN,NaN,12155.0,NaN,0.0,0.0,1.603066e+15
915992,14232.40,7788.8003,0.0,2662.4001,NaN,NaN,1500.6,60000.0,0.0,0.0,...,a55475b1,a55475b1,a55475b1,NaN,NaN,NaN,NaN,0.0,0.0,1.603066e+15
915993,197371.58,1195.4000,2827.2,8212.6010,NaN,47943.062,9921.2,6000.0,46806.6,46806.6,...,a55475b1,3439d993,a55475b1,NaN,NaN,15792.4,NaN,0.0,0.0,1.602893e+15


In [86]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(10)]
)

for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_train["target"], base_train["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')  

def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

stability_score_train = gini_stability(base_train)
stability_score_valid = gini_stability(base_valid)
stability_score_test = gini_stability(base_test)

print(f'The stability score on the train set is: {stability_score_train}') 
print(f'The stability score on the valid set is: {stability_score_valid}') 
print(f'The stability score on the test set is: {stability_score_test}')  

c:\python39\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[50]	valid_0's auc: 0.745397
[100]	valid_0's auc: 0.762503
[150]	valid_0's auc: 0.769837
[200]	valid_0's auc: 0.773977
[250]	valid_0's auc: 0.776265
[300]	valid_0's auc: 0.778061
[350]	valid_0's auc: 0.779516
[400]	valid_0's auc: 0.780804
[450]	valid_0's auc: 0.781701
[500]	valid_0's auc: 0.782555
[550]	valid_0's auc: 0.783098
[600]	valid_0's auc: 0.783914
[650]	valid_0's auc: 0.784483
[700]	valid_0's auc: 0.784923
Early stopping, best iteration is:
[710]	valid_0's auc: 0.784998
The AUC score on the train set is: 0.7977277095368003
The AUC score on the valid set is: 0.7849975220787444
The AUC score on the test set is: 0.7835577398875135
The stability score on the train set is: 0.5710330608208254
The stability score on the valid set is: 0.5399836334428477
The stability score on the test set is: 0.5269013067521229
